In [1]:
import pandas as pd

In [2]:
# Load the CSV files into DataFrames
patients = pd.read_csv("../MIMIC_III/PATIENTS.csv")
patients.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,1,2,M,2138-07-17 00:00:00,NaN,NaN,NaN,0
1,2,3,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1
2,4,5,M,2103-02-02 00:00:00,NaN,NaN,NaN,0
3,6,7,F,2121-05-23 00:00:00,NaN,NaN,NaN,0
4,7,8,M,2117-11-20 00:00:00,NaN,NaN,NaN,0


In [3]:
admissions = pd.read_csv('../MIMIC_III/ADMISSIONS.csv')
admissions.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1


In [4]:
admissions['diagnosis']

0                        NEWBORN
1                    HYPOTENSION
2                        NEWBORN
3                        NEWBORN
4                        NEWBORN
                  ...           
26831                    NEWBORN
26832                    NEWBORN
26833                    NEWBORN
26834    CORONARY ARTERY DISEASE
26835               BLUNT TRAUMA
Name: diagnosis, Length: 26836, dtype: object

In [9]:

diagnoses_icd = pd.read_csv('../MIMIC_III/DIAGNOSES_ICD.csv')
diagnoses_icd.columns = diagnoses_icd.columns.str.lower()

diagnoses_icd.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559


In [10]:
diagnoses_icd.shape

(225345, 5)

In [64]:
# Filter to get only the primary diagnosis for each admission
primary_diagnoses = diagnoses_icd[diagnoses_icd['seq_num'] == 1]
primary_diagnoses = primary_diagnoses.rename(columns={"icd9_code" : "primary_icd9_code"})
primary_diagnoses.head()

,row_id,subject_id,hadm_id,seq_num,primary_icd9_code
0,1,2,163353,1.0,V3001
3,4,3,145834,1.0,0389
12,22,5,178980,1.0,V3000
15,33,7,118037,1.0,V3001
18,36,8,159514,1.0,V3001


In [66]:
# Analyzing primary diagnoses frequencies
primary_diag_freq = primary_diagnoses['primary_icd9_code'].value_counts()
print(primary_diag_freq)

primary_icd9_code
V3000    3491
V3001    2746
41401    1603
V3101     997
41071     844
         ... 
78057       1
7761        1
76405       1
2794        1
1501        1
Name: count, Length: 1828, dtype: int64


In [67]:
# Merge admissions with patients on 'subject_id'
merged_data = pd.merge(admissions, patients, on='subject_id', how='inner')
merged_data.head(6)

,row_id_x,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,diagnosis,hospital_expire_flag,has_chartevents_data,row_id_y,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NEWBORN,0,1,1,M,2138-07-17 00:00:00,NaN,NaN,NaN,0
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,HYPOTENSION,0,1,2,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NEWBORN,0,1,4,M,2103-02-02 00:00:00,NaN,NaN,NaN,0
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NEWBORN,0,1,6,F,2121-05-23 00:00:00,NaN,NaN,NaN,0
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NEWBORN,0,1,7,M,2117-11-20 00:00:00,NaN,NaN,NaN,0
5,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,...,HEMORRHAGIC CVA,1,1,8,M,2108-01-26 00:00:00,2149-11-14 00:00:00,2149-11-14 00:00:00,2149-11-14 00:00:00,1


In [68]:
merged_data.columns

Index(['row_id_x', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'row_id_y', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag'],
      dtype='object')

In [69]:
# Further merge with diagnoses_icd on 'hadm_id'
# result = pd.merge(merged_data, diagnoses_icd, on='hadm_id', how='inner')
result = pd.merge(merged_data, primary_diagnoses, on='hadm_id', how='inner')

result.head()

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,row_id,subject_id_y,seq_num,primary_icd9_code
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,M,2138-07-17 00:00:00,NaN,NaN,NaN,0,1,2,1.0,V3001
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1,4,3,1.0,0389
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,M,2103-02-02 00:00:00,NaN,NaN,NaN,0,22,5,1.0,V3000
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,F,2121-05-23 00:00:00,NaN,NaN,NaN,0,33,7,1.0,V3001
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,M,2117-11-20 00:00:00,NaN,NaN,NaN,0,36,8,1.0,V3001


In [70]:
# Extract year directly from dates as integers
result['dob_year'] = pd.to_datetime(result['dob']).dt.year
result['admittime_year'] = pd.to_datetime(result['admittime']).dt.year

# Calculate age by subtracting the year directly
result['age'] = result['admittime_year'] - result['dob_year']


In [71]:
result['age']

0         0
1        76
2         0
3         0
4         0
         ..
26825     0
26826     0
26827     0
26828    81
26829    79
Name: age, Length: 26830, dtype: int32

In [72]:
# Assuming you've loaded your DataFrame as result
result['admittime'] = pd.to_datetime(result['admittime'])
result['dischtime'] = pd.to_datetime(result['dischtime'])

# Calculate the difference in hours (integer)
result['hospital_stay_hours'] = ((result['dischtime'] - result['admittime']).dt.total_seconds() / 3600).astype(int)
# Check for any negative values which might indicate data issues
# print("Negative hospital stays:", result[result['hospital_stay_hours'] < 0])

# You might want to set negative values to NaN or handle them accordingly
result.loc[result['hospital_stay_hours'] < 0, 'hospital_stay_hours'] = pd.NA
# Display some of the calculated hospital stays to verify
print(result[['admittime', 'dischtime', 'hospital_stay_hours']].head())


            admittime           dischtime  hospital_stay_hours
0 2138-07-17 19:04:00 2138-07-21 15:48:00                 92.0
1 2101-10-20 19:08:00 2101-10-31 13:58:00                258.0
2 2103-02-02 04:31:00 2103-02-04 12:15:00                 55.0
3 2121-05-23 15:05:00 2121-05-27 11:57:00                 92.0
4 2117-11-20 10:22:00 2117-11-24 14:20:00                 99.0


In [74]:
result.head()

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,dod_ssn,expire_flag,row_id,subject_id_y,seq_num,primary_icd9_code,dob_year,admittime_year,age,hospital_stay_hours
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,0,1,2,1.0,V3001,2138,2138,0,92.0
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2102-06-14 00:00:00,1,4,3,1.0,0389,2025,2101,76,258.0
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,0,22,5,1.0,V3000,2103,2103,0,55.0
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,0,33,7,1.0,V3001,2121,2121,0,92.0
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,0,36,8,1.0,V3001,2117,2117,0,99.0


In [75]:
# Drop duplicates if necessary
result = result.drop_duplicates()
result.columns

Index(['row_id_x', 'subject_id_x', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'row_id_y', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'row_id',
       'subject_id_y', 'seq_num', 'primary_icd9_code', 'dob_year',
       'admittime_year', 'age', 'hospital_stay_hours'],
      dtype='object')

In [107]:
columns_to_keep = [ 'gender', 'age', 'diagnosis', 'primary_icd9_code', 'hospital_expire_flag','expire_flag', 'hospital_stay_hours']

# Create a new DataFrame with only the selected columns

final_data = result[columns_to_keep]
final_data = final_data.rename(columns={"diagnosis" : "initial_diagnosis"})
final_data = final_data.dropna(axis=0)
final_data.head(20)



,gender,age,initial_diagnosis,primary_icd9_code,hospital_expire_flag,expire_flag,hospital_stay_hours
0,M,0,NEWBORN,V3001,0,0,92.0
1,M,76,HYPOTENSION,0389,0,1,258.0
2,M,0,NEWBORN,V3000,0,0,55.0
3,F,0,NEWBORN,V3001,0,0,92.0
4,M,0,NEWBORN,V3001,0,0,99.0
5,M,41,HEMORRHAGIC CVA,431,1,1,117.0
6,F,0,NEWBORN,V3000,0,0,192.0
7,F,50,BRAIN MASS,1913,0,1,612.0
8,M,72,PANCREATIC CANCER/SDA,1570,1,1,304.0
9,F,40,CORONARY ARTERY DISEASE,41401,0,0,164.0


In [108]:
final_data.dtypes

gender                   object
age                       int32
initial_diagnosis        object
primary_icd9_code        object
hospital_expire_flag      int64
expire_flag               int64
hospital_stay_hours     float64
dtype: object

In [109]:
final_data[final_data['hospital_stay_hours'].isna()]

,gender,age,initial_diagnosis,primary_icd9_code,hospital_expire_flag,expire_flag,hospital_stay_hours


In [112]:
# Converting data types
final_data['gender'] = final_data['gender'].astype('str')
final_data['initial_diagnosis'] = final_data['initial_diagnosis'].astype('str')
final_data['primary_icd9_code'] = final_data['primary_icd9_code'].astype('str')
final_data['hospital_expire_flag'] = final_data['hospital_expire_flag'].astype('boolean')
final_data['expire_flag'] = final_data['expire_flag'].astype('boolean')
final_data['hospital_stay_hours'] = final_data['hospital_stay_hours'].astype('int32')

In [113]:
# Filter to include only diabetes-related records
# For example, ICD-9 codes starting with '250' indicate diabetes
diabetes_data = final_data[final_data['primary_icd9_code'].str.startswith('250', na=False)]
diabetes_data.head(-1)

,gender,age,initial_diagnosis,primary_icd9_code,hospital_expire_flag,expire_flag,hospital_stay_hours
220,M,56,PERIPHERAL INSUFFICIENCY;RIGHT FOOT,25073,False,True,284
241,F,86,INFECTED RIGHT FOOT,25060,False,True,43
293,F,73,ENCEPHALOPATHY;TELEMETRY,25011,False,True,245
564,M,50,BILIARY COLIC,25080,False,False,46
656,M,32,DIABETIC KETOACIDOSIS,25011,False,False,92
...,...,...,...,...,...,...,...
24315,F,64,DIABETIC KETOACIDOSIS,25011,True,True,118
24769,M,74,FOOT INFECTION,25060,False,False,428
25007,M,54,DIABETIC KETOACIDOSIS,25012,False,False,159
25375,F,54,METABOLIC ACIDOSIS,25012,False,True,249


In [117]:
diabetes_data.dtypes

gender                  category
age                        int32
initial_diagnosis         object
primary_icd9_code         object
hospital_expire_flag     boolean
expire_flag              boolean
hospital_stay_hours        int32
dtype: object

In [116]:
final_data.to_csv('../Notebooks/test_datasets/processed_data.csv', index=False)
diabetes_data.to_csv('../Notebooks/test_datasets/processed_diabetes_data.csv', index=False)